In [32]:
class ARGS():
    def __init__(self):
        self.gpus = '0'
        self.cpus = 4
        self.seed = 42
        self.network = 'GEM'
        self.n_tasks = 5
        self.n_memories = 256
        self.memory_strength = 0.5
        self.model = 'ResNet18'
        self.channel = 64
        self.load = False
        self.official = False
        self.checkpoint = 'checkpoints'
        self.epochs = 5
        self.loss = 'CE'
        self.init_lr = 1e-2
        self.lr_scheduler = 'cosine_annealing'
        self.optimizer = 'Adam'
        self.eval_term = 1
        self.dataset = 'mnist'
        self.continuum = True
        self.data_path = 'data'
        self.image_size = [256, 256]
        self.batch_size = 10
        self.in_channel = 3
args = ARGS()

In [52]:
from network.utils import load_network
import torch
import numpy as np

device = 'cpu'
network, start_epoch, best_recall, recalls, save_path = load_network(args, 10, device=device)

Create ResNet18 model


In [53]:
network.grad_dims[:10]

[9408, 64, 64, 64, 36864, 64, 64, 64, 36864, 64]

In [54]:
network.grads.shape

torch.Size([11186442, 5])

In [55]:
task = 2
network.grads[:, task].unsqueeze(0).shape

torch.Size([1, 11186442])

In [56]:
idx = torch.LongTensor([i for i in range(task)])
network.grads.index_select(1, idx).shape

torch.Size([11186442, 2])

In [57]:
torch.mm(network.grads[:, task].unsqueeze(0),
         network.grads.index_select(1, idx))

tensor([[0., 0.]])

In [58]:
network.grads[:, 0].unsqueeze(1).shape

torch.Size([11186442, 1])

In [89]:
g=torch.Tensor(10).numpy()
m=torch.Tensor(10,3).t().numpy()

In [100]:
margin = args.memory_strength

In [101]:
P = np.dot(m, m.transpose())
t = m.shape[0]

In [102]:
P = 0.5 * (P + P.transpose()) + np.eye(t) * 1e-6
P.shape

(3, 3)

In [103]:
q = np.dot(m, g) * -1
q.shape

(3,)

In [104]:
G = np.eye(t)
G.shape

(3, 3)

In [106]:
h = np.zeros(t) + margin